<font color='006A58'>**Social Network Graphs**</font>

Social media represents, nowadays, one of the most interesting and rich sources of information. Every day, thousands of new connections arise, new users join communities, and billions of posts are shared. Graphs mathematically represent all those interactions, helping to make order of all such spontaneous and unstructured traffic.

When dealing with social graphs, there are many interesting problems that can be addressed using machine learning. Under the correct settings, it is possible to extract useful insights from this huge amount of data, for improving your marketing strategy, identifying users with dangerous behaviors (for example, terrorist networks), and predicting the likelihood that a user will read your new post.

Specifically, link prediction is one of the most interesting and important research topics in this field. Depending on what a connection in your social graph represents, by predicting future edges, you will be able to predict your next suggested friend, the next suggested movie, and which product you are likely to buy.

> Tip: Link prediction task aims at forecasting the likelihood of a future connection between two nodes and it can be solved using several machine learning algorithms.

In [ ]:
!wget http://snap.stanford.edu/data/facebook_combined.txt.gz
!wget http://snap.stanford.edu/data/facebook.tar.gz
!gzip -d facebook_combined.txt.gz
!tar -xf facebook.tar.gz

In [ ]:
!pip install community
!pip install stellargraph
!pip install node2vec==0.3.3
!pip install git+https://github.com/palash1992/GEM.git

In [12]:
import os
import math
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 

import community
from community import community_louvain
import networkx as nx
import networkx.algorithms.community as nx_comm
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder 
from stellargraph.data import EdgeSplitter
from stellargraph import StellarGraph
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from tensorflow import keras

%matplotlib inline

default_edge_color = 'gray'
default_node_color = '#407cc9'
enhanced_node_color = '#f5b042'
enhanced_edge_color = '#cc2f04'

Our goal is to predict the probability that the given 2 nodes are connected. It comes under the *Link Prediction* task. For this, we will first preprocess the data in the right format and then split it into train/test. Then, we will try 3 methods to train a binary classifier that will take 2 nodes as input and outputs the probability.

## Preprocessing

### Parse edge features

In [2]:
feat_file_name = "feature_map.txt"
feature_index = {}  #numeric index to name
inverted_feature_index = {} #name to numeric index
network = nx.Graph()

In [3]:
G = nx.read_edgelist("facebook_combined.txt", create_using=nx.Graph(), nodetype=int)
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 4039
Number of edges: 88234
Average degree:  43.6910


In [4]:
# let's first create a list of participant ids - we call it ego nodes in literature it seems
ego_nodes = set([int(name.split('.')[0]) for name in os.listdir("./facebook/")])

In [5]:
def parse_featname_line(line):
  """ used to parse each line of the files containing feature names """
  line = line[(line.find(' '))+1:]  # chop first field
  split = line.split(';')
  name = ';'.join(split[:-1]) # feature name
  index = int(split[-1].split(" ")[-1]) #feature index
  return index, name

In [6]:
def load_features():
  """ 
  parse each ego-network and creates two dictionaries:
      - feature_index: maps numeric indices to names
      - inverted_feature_index: maps names to numeric indices
  """
  feat_file_name = 'tmp.txt'
  # may need to build the index first
  if not os.path.exists(feat_file_name):
      feat_index = {}
      # build the index from data/*.featnames files
      featname_files = glob.iglob("facebook/*.featnames")
      for featname_file_name in featname_files:
          featname_file = open(featname_file_name, 'r')
          for line in featname_file:
              # example line:
              # 0 birthday;anonymized feature 376
              index, name = parse_featname_line(line)
              feat_index[index] = name
          featname_file.close()
      keys = feat_index.keys()
      keys = sorted(keys)
      out = open(feat_file_name,'w')
      for key in keys:
          out.write("%d %s\n" % (key, feat_index[key]))
      out.close()

  index_file = open(feat_file_name,'r')
  for line in index_file:
      split = line.strip().split(' ')
      key = int(split[0])
      val = split[1]
      feature_index[key] = val
  index_file.close()

  for key in feature_index.keys():
      val = feature_index[key]
      inverted_feature_index[val] = key

In [7]:
load_features()

### Add parsed feature to the node

In [8]:
def parse_nodes(network, ego_nodes):
  """
  for each nodes in the network assign the corresponding features 
  previously loaded using the load_features function
  """
  # parse each node
  for node_id in ego_nodes:
      featname_file = open(f'facebook/{node_id}.featnames','r')
      feat_file     = open(f'facebook/{node_id}.feat','r')
      egofeat_file  = open(f'facebook/{node_id}.egofeat','r')
      edge_file     = open(f'facebook/{node_id}.edges','r')

      ego_features = [int(x) for x in egofeat_file.readline().split(' ')]

      # Add ego node features
      network.nodes[node_id]['features'] = np.zeros(len(feature_index))
      
      # parse ego node
      i = 0
      for line in featname_file:
          key, val = parse_featname_line(line)
          # Update feature value if necessary
          if ego_features[i] + 1 > network.nodes[node_id]['features'][key]:
              network.nodes[node_id]['features'][key] = ego_features[i] + 1
          i += 1

      # parse neighboring nodes
      for line in feat_file:
          featname_file.seek(0)
          split = [int(x) for x in line.split(' ')]
          node_id = split[0]
          features = split[1:]

          # Add node features
          network.nodes[node_id]['features'] = np.zeros(len(feature_index))

          i = 0
          for line in featname_file:
              key, val = parse_featname_line(line)
              # Update feature value if necessary
              if features[i] + 1 > network.nodes[node_id]['features'][key]:
                  network.nodes[node_id]['features'][key] = features[i] + 1
              i += 1
          
      featname_file.close()
      feat_file.close()
      egofeat_file.close()
      edge_file.close()

In [9]:
# add the parsed features to the networkx nodes
parse_nodes(G, ego_nodes)

In [10]:
# check features has been correctly assigned
G.nodes[0]

{'features': array([1., 1., 1., ..., 0., 0., 0.])}

### Data split

Since we aim to cast this problem as a supervised learning task, we need to create a training and testing dataset. We will therefore create two new subgraphs with the same numbers of nodes but different numbers of edges (as some edges will be removed and treated as positive samples for training/testing the algorithm).

We are using the EdgeSplitter class to extract a fraction (p=10%) of all the edges in G, as well as the same number of negative edges, in order to obtain a reduced graph, graph_test. The train_test_split method also returns a list of node pairs, samples_test (where each pair corresponds to an existing or not existing edge in the graph), and a list of binary targets (labels_test) of the same length of the samples_test list. Then, from such a reduced graph, we are repeating the operation to obtain another reduced graph, graph_train, as well as the corresponding samples_train and labels_train lists.

In [11]:
edgeSplitter = EdgeSplitter(G) 
graph_test, samples_test, labels_test = edgeSplitter.train_test_split(p=0.1, method="global", seed=24)

edgeSplitter = EdgeSplitter(graph_test, G) 
graph_train, samples_train, labels_train = edgeSplitter.train_test_split(p=0.1, method="global", seed=24)

** Sampled 8823 positive and 8823 negative edges. **
** Sampled 7941 positive and 7941 negative edges. **


## Modeling

We will be comparing three different methods for predicting missing edges:
- Method1: node2vec will be used to learn a node embedding. Such embeddings will be used to train a Random Forest classifier in a supervised manner
- Method2: graphSAGE (with and without features) will be used for link prediction
- Method3: hand-crafted features will be extracted and used to train a Random Forest classifier

### Node2vec

In [16]:
node2vec = Node2Vec(graph_train) 
model = node2vec.fit() 

edges_embs = HadamardEmbedder(keyed_vectors=model.wv) 
train_embeddings = [edges_embs[str(x[0]),str(x[1])] for x in samples_train]

edges_embs = HadamardEmbedder(keyed_vectors=model.wv) 
test_embeddings = [edges_embs[str(x[0]),str(x[1])] for x in samples_test]

Generating walks (CPU: 1): 100%|██████████| 10/10 [02:44<00:00, 16.48s/it]


In [18]:
rf = RandomForestClassifier(n_estimators=10) 
rf.fit(train_embeddings, labels_train); 
 
y_pred = rf.predict(test_embeddings) 
print('Precision:', metrics.precision_score(labels_test, y_pred)) 
print('Recall:', metrics.recall_score(labels_test, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_test, y_pred)) 

Precision: 0.9660746812386156
Recall: 0.9618043749291624
F1-Score: 0.9639347986596241


### GraphSage with no features

We will build a two-layer GraphSAGE architecture that, given labeled pairs of nodes, outputs a pair of node embeddings. Then, a fully connected neural network will be used to process these embeddings and produce link predictions. Notice that the GraphSAGE model and the fully connected network will be concatenated and trained end to end so that the embeddings learning stage is influenced by the predictions.

In [19]:
eye = np.eye(graph_train.number_of_nodes())
fake_features = {n:eye[n] for n in G.nodes()}
nx.set_node_attributes(graph_train, fake_features, "fake")

eye = np.eye(graph_test.number_of_nodes())
fake_features = {n:eye[n] for n in G.nodes()}
nx.set_node_attributes(graph_test, fake_features, "fake")

In [21]:
graph_train.nodes[0]

{'fake': array([1., 0., 0., ..., 0., 0., 0.]),
 'features': array([1., 1., 1., ..., 0., 0., 0.])}

In [22]:
batch_size = 64
num_samples = [4, 4]

sg_graph_train = StellarGraph.from_networkx(graph_train, node_features="fake")
sg_graph_test = StellarGraph.from_networkx(graph_test, node_features="fake")

train_gen = GraphSAGELinkGenerator(sg_graph_train, batch_size, num_samples)
train_flow = train_gen.flow(samples_train, labels_train, shuffle=True, seed=24)

test_gen = GraphSAGELinkGenerator(sg_graph_test, batch_size, num_samples)
test_flow = test_gen.flow(samples_test, labels_test, seed=24)

In [24]:
layer_sizes = [20, 20]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

x_inp, x_out = graphsage.in_out_tensors()

prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.mse,
    metrics=["acc"],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [25]:
epochs = 10
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow)

Epoch 1/10
249/249 [==============================] - 67s 260ms/step - loss: 0.2299 - acc: 0.6080 - val_loss: 0.1921 - val_acc: 0.7242
Epoch 2/10
249/249 [==============================] - 64s 258ms/step - loss: 0.1957 - acc: 0.7239 - val_loss: 0.1837 - val_acc: 0.7444
Epoch 3/10
249/249 [==============================] - 65s 259ms/step - loss: 0.1863 - acc: 0.7495 - val_loss: 0.1789 - val_acc: 0.7587
Epoch 4/10
249/249 [==============================] - 65s 260ms/step - loss: 0.1814 - acc: 0.7656 - val_loss: 0.1764 - val_acc: 0.7814
Epoch 5/10
249/249 [==============================] - 65s 260ms/step - loss: 0.1775 - acc: 0.7877 - val_loss: 0.1758 - val_acc: 0.7902
Epoch 6/10
249/249 [==============================] - 64s 257ms/step - loss: 0.1754 - acc: 0.7957 - val_loss: 0.1746 - val_acc: 0.7944
Epoch 7/10
249/249 [==============================] - 64s 258ms/step - loss: 0.1734 - acc: 0.7985 - val_loss: 0.1741 - val_acc: 0.7993
Epoch 8/10
249/249 [==============================] - 6

In [26]:
y_pred = np.round(model.predict(train_flow)).flatten()
print('Precision:', metrics.precision_score(labels_train, y_pred)) 
print('Recall:', metrics.recall_score(labels_train, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_train, y_pred)) 

Precision: 0.5053176470588235
Recall: 0.6761113209923183
F1-Score: 0.5783690617257352


In [27]:
y_pred = np.round(model.predict(test_flow)).flatten()
print('Precision:', metrics.precision_score(labels_test, y_pred)) 
print('Recall:', metrics.recall_score(labels_test, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_test, y_pred)) 

Precision: 0.7233901671908625
Recall: 0.9905927688994673
F1-Score: 0.8361635972255441


### GraphSage with features

In [28]:
sg_graph_train = StellarGraph.from_networkx(graph_train, node_features="features")
sg_graph_test = StellarGraph.from_networkx(graph_test, node_features="features")

train_gen = GraphSAGELinkGenerator(sg_graph_train, batch_size, num_samples)
train_flow = train_gen.flow(samples_train, labels_train, shuffle=True, seed=24)

test_gen = GraphSAGELinkGenerator(sg_graph_test, batch_size, num_samples)
test_flow = test_gen.flow(samples_test, labels_test, seed=24)

In [31]:
layer_sizes = [20, 20]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

x_inp, x_out = graphsage.in_out_tensors()

prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.mse,
    metrics=["acc"],
)

epochs = 10
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow)

link_classification: using 'ip' method to combine node embeddings into edge embeddings
Epoch 1/10
249/249 [==============================] - 61s 240ms/step - loss: 0.1773 - acc: 0.8178 - val_loss: 0.1671 - val_acc: 0.8495
Epoch 2/10
249/249 [==============================] - 31s 123ms/step - loss: 0.1702 - acc: 0.8561 - val_loss: 0.1666 - val_acc: 0.8629
Epoch 3/10
249/249 [==============================] - 31s 124ms/step - loss: 0.1684 - acc: 0.8572 - val_loss: 0.1666 - val_acc: 0.8642
Epoch 4/10
249/249 [==============================] - 30s 122ms/step - loss: 0.1670 - acc: 0.8594 - val_loss: 0.1665 - val_acc: 0.8609
Epoch 5/10
249/249 [==============================] - 31s 124ms/step - loss: 0.1666 - acc: 0.8714 - val_loss: 0.1666 - val_acc: 0.8607
Epoch 6/10
249/249 [==============================] - 31s 123ms/step - loss: 0.1664 - acc: 0.8692 - val_loss: 0.1666 - val_acc: 0.8752
Epoch 7/10
249/249 [==============================] - 31s 124ms/step - loss: 0.1659 - acc: 0.8745 - val

In [32]:
y_pred = np.round(model.predict(train_flow)).flatten()
print('Precision:', metrics.precision_score(labels_train, y_pred)) 
print('Recall:', metrics.recall_score(labels_train, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_train, y_pred)) 

Precision: 0.5031298101590559
Recall: 0.617428535448936
F1-Score: 0.5544498473368765


In [33]:
y_pred = np.round(model.predict(test_flow)).flatten()
print('Precision:', metrics.precision_score(labels_test, y_pred)) 
print('Recall:', metrics.recall_score(labels_test, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_test, y_pred)) 

Precision: 0.8077488064634594
Recall: 0.9971664966564661
F1-Score: 0.8925183870149632


### Hand-crafted features

In [34]:
def get_shortest_path(G,u,v):
  """ return the shortest path length between u,v 
      in the graph without the edge (u,v) """
  removed = False
  if G.has_edge(u,v):
    removed = True
    G.remove_edge(u,v) # temporary remove edge
  
  try:
    sp = len(nx.shortest_path(G, u, v))
  except:
    sp = 0

  if removed:
    G.add_edge(u,v) # add back the edge if it was removed

  return sp

In [38]:
def get_hc_features(G, samples_edges, labels):
  # precompute metrics
  centralities = nx.degree_centrality(G)
  parts = community_louvain.best_partition(G)
  
  feats = []
  for (u,v),l in zip(samples_edges, labels):
    shortest_path = get_shortest_path(G, u, v)
    j_coefficient = next(nx.jaccard_coefficient(G, ebunch=[(u, v)]))[-1]
    u_centrality = centralities[u]
    v_centrality = centralities[v]
    u_community = parts.get(u)
    v_community = parts.get(v)
    # add the feature vector
    feats += [[shortest_path, j_coefficient, u_centrality, v_centrality]]
  return feats

In [39]:
feat_train = get_hc_features(graph_train, samples_train, labels_train)
feat_test = get_hc_features(graph_test, samples_test, labels_test)

In [40]:
rf = RandomForestClassifier(n_estimators=10) 
rf.fit(feat_train, labels_train); 
 
y_pred = rf.predict(feat_test) 
print('Precision:', metrics.precision_score(labels_test, y_pred)) 
print('Recall:', metrics.recall_score(labels_test, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_test, y_pred)) 

Precision: 0.9650459332287699
Recall: 0.9763119120480562
F1-Score: 0.9706462335906249


## Summary of results

In [ ]:
results = pd.DataFrame(columns=['Algorithm','Embedding','Node Features',
                                'Precision','Recall','F1-Score'])

idx = 0
while True:
    for col in results.columns:
        _col = input(f"{col}: ")
        if _col=='\stop': break
        results.loc[idx,col] = _col
    print('\n{}\n'.format('='*100))
    if _col=='\stop': break
    idx+=1

In [59]:
results

,Algorithm,Embedding,Node Features,Precision,Recall,F1-Score
0,node2vec,Unsupervised,No,0.97,0.96,0.96
1,GraphSAGE,Supervised,No,0.72,0.99,0.84
2,GraphSAGE,Supervised,Yes,0.81,0.99,0.89
3,Shallow,Manual,No,0.97,0.98,0.97


The node2vec-based method is already able to achieve a high level of performance without supervision and per-node information. Such high results might be related to the particular structure of the combined ego network. Due to the high sub-modularity of the network (since it is composed of several ego networks), predicting whether two users will be connected or not might be highly related to the way the two candidate nodes are connected inside the network. For example, there might be a systematic situation in which two users, both connected to several users in the same ego network, have a high chance of being connected as well. On the other hand, two users belonging to different ego networks, or very far from each other, are likely to not be connected, making the prediction task easier. This is also confirmed by the high results achieved using the shallow method.

Such a situation might be confusing, instead, for more complicated algorithms like GraphSAGE, especially when node features are involved. For example, two users might share similar interests, making them very similar. However, they might belong to different ego networks, where the corresponding ego users live in two very different parts of the world. So, similar users, which in principle should be connected, are not. However, it is also possible that such algorithms are predicting something further in the future. Recall that the combined ego network is a timestamp of a particular situation in a given period of time. Who knows how it might have evolved right now!

Interpreting machine learning algorithms is probably the most interesting challenge of machine learning itself. For this reason, we should always interpret results with care. Our suggestion is always to dig into the dataset and try to give an explanation of your results.

Finally, it is important to remark that each of the algorithms was not tuned for the purpose of this demonstration. Different results can be obtained by properly tuning each hyperparameter.